# Predicting Whether an Instagram Account is Fake or Genuine

Author: *Khoi Ngo*

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Overview of Steps to Accomplish Notebook

## 0. Function Definitions

In [ ]:
from sklearn.datasets import load_files

def load_train_data():
    ''' Load data from csv files, Train data 
        Splits data into X (feature matrix) and y (labels)
        
        returns: X_train, y_train
        
    '''
    train_data = pd.read_csv('../input/instagram-fake-spammer-genuine-accounts/train.csv', header = 0)
    
    X_train = train_data.drop(columns='fake')
    y_train = train_data['fake']
    
    return X_train, y_train

In [ ]:
from sklearn.datasets import load_files

def load_test_data():
    ''' Load data from csv files, Train and Test data 
        Splits data into X (feature matrix) and y (labels)
        
        returns: X_test, y_test
        
    '''
    test_data = pd.read_csv('../input/instagram-fake-spammer-genuine-accounts/test.csv', header = 0)
    
    X_test = test_data.drop(columns='fake')
    y_test = test_data['fake']
    
    return X_test, y_test

In [ ]:
from sklearn.model_selection import cross_validate


def get_classifier_cv_score(model, X, y, scoring='accuracy', cv=7):
    '''Calculate train and validation score of classifier (model) using cross-validation
        
        
        model (sklearn classifier): Classifier to train and evaluate
        X (numpy.array or pandas.DataFrame): Feature matrix
        y (numpy.array or pandas.Series): Target vector
        scoring (str): a scoring string accepted by sklearn.metrics.cross_validate()
        cv (int): number of cross-validation folds see sklearn.metrics.cross_validate()
        
        returns: mean training score, mean validation score
    
    '''
    scores = cross_validate(model, X, y, cv=cv, scoring=scoring, return_train_score=True)
    train_scores = scores['train_score']
    val_scores = scores['test_score']
    
    train_mean = np.mean(train_scores)
    val_mean = np.mean(val_scores)
    
    return train_mean, val_mean

In [ ]:
def print_grid_search_result(grid_search):
    '''Prints best parameters and mean training and validation scores of a grid search object.
    
        grid_search (sklearn GridSearchCV): Fitted GridSearchCV object
        
        scores are printed with 3 decimal places.
        
    '''
    
     #TODO: implement function body
    
    print(grid_search.best_params_)
    
    best_train = grid_search.cv_results_["mean_train_score"][grid_search.best_index_]
    print("best mean_train_score: {:.3f}".format(best_train))
        
    best_test = grid_search.cv_results_["mean_test_score"][grid_search.best_index_]
    print("best mean_test_score: {:.3f}".format(best_test))

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_actual, y_pred, labels, title=''):
    '''Creates a heatmap plot of the confusion matrix.
    
        y_actual (pandas.DataSeries or numpy.Array): Ground truth label vector
        y_pred (pandas.DataSeries or numpy.Array): Predicted label vector
        labels (list(str)): Class names used for plotting (ticklabels)
        title (str): Plot title
        
        uses sklearn.metrics.confusion_matrix
        
    '''
    data = confusion_matrix(y_actual, y_pred)
    ax = sns.heatmap(data,
                     annot=True,
                     cbar=False,
                     fmt='d',
                     xticklabels = labels,
                     yticklabels = labels)
    ax.set_title(title)
    ax.set_xlabel("predicted values")
    ax.set_ylabel("actual values")

## 1. Load Data

In [ ]:
X_data, y_data = load_train_data()
print(X_data.info())

In [ ]:
X_data.head()

In [ ]:
print("Size: ",X_data.shape, ", Type: ", type(X_data))
print("Size: ",y_data.shape, ", Type: ", type(y_data))

## 2. Inspect Data

### 2.1 Seeing Any Correlation in Features
- We can see that there is no correlation among the features
- They are roughly around 0 in each feature comparison.

In [ ]:
data_corr = X_data.corr(method='pearson')
ax = sns.heatmap(data_corr, vmin=-1, vmax=1, cmap='BrBG')
ax.set_title("Correlation Heatmap Between Features")

### 2.2 Missing any Values
- We can see that the features are all filled out and we do not need to modify the existing data

In [ ]:
print(X_data.isnull().sum())

### 2.3 Checking if Imbalance in Labels
- The labels is about 1:1 which means there is no imbalance in the labels. 
- If there was, the ratio would be more 2:1.

In [ ]:
unique, freq = np.unique(y_data, return_counts = True) 

for i, j in zip(unique, freq):
    print("Label: ", i, ", Frequency: ", j)

## 3. Create training and test sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=37)

In [ ]:
print(X_train.shape)
print(y_train.shape)

## 4. Compare Models Using Cross-Validation

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

model_list = [LogisticRegression(max_iter=600),
              SVC(), 
              GaussianNB(),
              RandomForestClassifier(random_state=55),
              GradientBoostingClassifier(random_state=56)]

train_scores = []
val_scores = []

for model in model_list:
    train, val = get_classifier_cv_score(model, X_train, y_train,'average_precision')
    train_scores.append(train)
    val_scores.append(val)
    
models_score = sorted(list(zip(val_scores, train_scores, model_list)), reverse=True)

print("-------------------------------------")
for val, train, model in models_score:
    print("Model: {} ".format(model.__class__.__name__))

    print("train_score: {:.3f}".format(train)) 

    print("validation_score: {:.3f}".format(val)) 

    print("-------------------------------------")

- We will select the two best models to continue.
- These models are Random Forest Classifier and Gradient Boosting Classifier.

## 5. Hyperparameter Tuning Using Grid Search

### 5.1 Grid Search for RandomForestClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
import os
#if run on own computer
#num_cpu = int(os.environ['NUMBER_OF_PROCESSORS'])

model = RandomForestClassifier(random_state=55)

parameters = {'n_estimators': [300, 500, 700, 1000],
              'max_depth': [7, 9, 11, 13]}

#if run on own computer
#grid1 = GridSearchCV(model, parameters, cv=7, scoring='average_precision', n_jobs=num_cpu, return_train_score=True)
grid1 = GridSearchCV(model, parameters, cv=7, scoring='average_precision',return_train_score=True)

In [ ]:
grid1.fit(X_train, y_train)

In [ ]:
print_grid_search_result(grid1)

### 5.2 Grid Search for Gradient Boosting Classifier

In [ ]:
model = GradientBoostingClassifier(max_depth=5, random_state=56)

parameters = {'n_estimators': [50, 100, 200],
              'learning_rate': [0.001, 0.01, 0.1, 1.0, 10.0]}

#if run on own computer
#grid2 = GridSearchCV(model, parameters, cv=7, scoring='average_precision', n_jobs=num_cpu, return_train_score=True)
grid2 = GridSearchCV(model, parameters, cv=7, scoring='average_precision', return_train_score=True)

In [ ]:
grid2.fit(X_train, y_train)

In [ ]:
print_grid_search_result(grid2)

## 6. Pipeline

- Now we will use the best model from our grid search, which is Random Forest Classifier
- We will apply a pipeline since we need to standardize our data

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([('preprocessing', StandardScaler()), ('classifier', grid1.best_estimator_)])
pipeline.fit(X_train, y_train)

In [ ]:
print("Test score: {:.3f}".format(pipeline.score(X_test, y_test)))

## 7. Final Evaluation

- Finally we have a working model to do a final evaluation on a reserve data

In [ ]:
X_final, y_final = load_test_data()

In [ ]:
print("Test score: {:.3f}".format(pipeline.score(X_final, y_final)))

In [ ]:
from sklearn.metrics import classification_report
y_pred = pipeline.predict(X_final)
print(classification_report(y_final, y_pred, target_names=["genuine", "fake"]))

In [ ]:
labels = ["genuine", "fake"]
title = "Predicting Fake Instagram Account"
plot_confusion_matrix(y_final, y_pred, labels, title)

- We can see our model predicted around 91.5% fake accounts and 90.2% genuine accounts correctly
- The model only predicted 11 accounts wrong